# Self-Driving Car Engineer Nanodegree


## Project \#1: **Finding Lane Lines on the Road**    
(built by Hector Angulo May 2017)


## Import Packages

In [5]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

## Helper Functions

In [60]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    left_slopes = []
    right_slopes = []
    left_coordinates = []
    right_coordinates = []
    min_y  =image.shape[0]  # Want to find the farthest point away from car to have both lines have same length.

    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1))

            ## Lets group into left and right lane
            if slope > .1 and slope < 100:   # Right lane since image is inverted
                gr_slopes.append(slope)
                right_coordinates.append([x1,y1])
                right_coordinates.append([x2,y2])
              
            elif slope < -.1 and slope > -100:  #Left lane
                gl_slopes.append(slope)
                left_coordinates.append([x1,y1])
                left_coordinates.append([x2,y2])
                
            else: 
                print("### Too close to horizontal or  vertical line. Removing from group ####")
                           
    # TODO: This is very inneficient memory-wise, dont need ALL the entries since list gets into thousands after
    ## just a few seconds of video, a rolling average is enough. But this proves we can smooth out lines with 
    ## global memory
    
    average_left = sum(gl_slopes)/ len(gl_slopes)
    average_right = sum(gr_slopes)/ len(gr_slopes)
    
    print(len(gr_slopes))

    # Sort the coordinates to better find the leftmost and rightmost points (i.e extreme of lines)
    # Got this sorting tip off of Stackoverflow.  Dont fully understand the 'lambda' context but seems its a way to
    # define HOW you want things sorted and here I'm saying sort by X coordinate first, then Y.
    sorted_left = sorted(left_coordinates , key=lambda k: [k[0], k[1]])  
    sorted_right = sorted(right_coordinates , key=lambda k: [k[0], k[1]])
    
    
    # We will calculate the 'b' for each line and also find the extreme coordinates to create a new single line
    left_b= sorted_left[0][1] - average_left*sorted_left[0][0]      # calculate b for left line
    left_bottom_x = int((img.shape[0]-left_b)/average_left)
    last_coordinate = len(sorted_left)-1  #find the  coordinate for the 'top' of left line
    
    right_b = sorted_right[0][1] - average_right*sorted_right[0][0] 
    right_bottom_x = int((img.shape[0]-right_b)/average_right)
    
    left_y = sorted_left[last_coordinate][1]
    right_y = sorted_right[0][1]
    min_y = min(left_y, right_y)
    
    left_top_x= int((min_y-left_b)/average_left)
    right_top_x = int((min_y-right_b)/average_right)
    
    # Define the ends of both lines
    bottom_left = (left_bottom_x, img.shape[0])
    #top_left  = (left_top_x,min_y)   
    ### older code
    last_coordinate = len(sorted_left)-1  #find the  coordinate for the 'top' of left line
    top_left  = (sorted_left[last_coordinate][0],sorted_left[last_coordinate][1])   
    
    bottom_right = (right_bottom_x, img.shape[0])
    top_right = (sorted_right[0][0],min_y)     

    #print(average_left, left_b, left_top_x, bottom_left, top_left)
    
    # Now draw the lines with all the info we have
    cv2.line(img, bottom_left, top_left, color, thickness)   #Left line
    
    cv2.line(img, bottom_right, top_right, color, thickness)#Right line
    
    
def draw_lines_orig(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Lane Finding Pipeline

In [52]:
def process_image(image):
    gray = grayscale(image)
    img_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    ##  Mask for only white and yellow lines
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(gray, 200, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yw_image = cv2.bitwise_and(gray, mask_yw)
                            
    ## Blur to reduce noise before doing edge detection
    gauss = cv2.GaussianBlur(mask_yw_image,(5,5), 0)

    # Do Canny Edge Detection   
    low_threshold = 50
    high_threshold = 150
    canny_edges = canny(gauss,low_threshold,high_threshold)

    # Region of Interest
    vertices = [np.array([[80,540],[465,315],[500,315],[900,540]], dtype=np.int32)]
    ROI_masked = region_of_interest(canny_edges, vertices)

    # Hough Transform
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_len = 25
    max_line_gap = 50
    hough_image = hough_lines(ROI_masked, rho, theta, threshold, min_line_len, max_line_gap)

    # Apply the lines to the original image
    weighted_image = weighted_img(hough_image, image, α=.8, β=1., λ=0.)
    
    return weighted_image

## Test it on an Image

In [61]:
global gl_slopes
gl_slopes = []
global gr_slopes
gr_slopes = []
image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')

print("average_left, left_b, left_top_x, bottom_left, top_left")
final_image = process_image(image)
plt.figure(figsize=(20,10))
plt.imshow(final_image)

average_left, left_b, left_top_x, bottom_left, top_left


NameError: name 'gr_slopes' is not defined

## Test it on Video

In [59]:
global gl_slopes
gl_slopes = []
gr_slopes = []

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
print("average_left, left_b, left_top_x, bottom_left, top_left")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

average_left, left_b, left_top_x, bottom_left, top_left
8
[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


  5%|▍         | 11/222 [00:00<00:02, 102.97it/s]

16
21
29
36
39
42
44
47
50
54
58
62
69
77
85
90
94
97
101
105
110


 15%|█▍        | 33/222 [00:00<00:01, 104.34it/s]

114
122
127
136
143
154
161
164
166
169
172
177
182
189
194
201
209
217
224
228
231
234

...

1015
1017
1022
1026
1030
1034
1039
1045
1051
1057
1065
1078
1085
1088
1091


 94%|█████████▎| 208/222 [00:02<00:00, 72.13it/s]

1095
1099
1103
1109
1116
1123
1129
1136
1145
1153
1155
1158
1161
1165
1170
1174


100%|█████████▉| 221/222 [00:03<00:00, 73.30it/s]


1180
1186
1193
1200
1207
1213
[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 3.58 s, sys: 683 ms, total: 4.26 s
Wall time: 3.42 s
